<a href="https://colab.research.google.com/github/Sheryar-bit/Lang-Chain/blob/main/fine_tuning_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

Image1 = pipe(prompt="black hoodie with a front half zipper by martin valen").images[0]
Image1.save("ProductSnapAI_panel_1.png")

Image2 = pipe(prompt="white hoodie with a blue design by martin valen").images[0]
Image2.save("ProductSnapAI_panel_2.png")

Image3 = pipe(prompt="stripped hoodie by martin valen").images[0]
Image3.save("ProductSnapAI_panel_3.png")

Image4 = pipe(prompt="camouflage hoodie by martin valen").images[0]
Image4.save("ProductSnapAI_panel_4.png")

# Image grid helper function from HuggingFace
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=((i % cols) * w, (i // cols) * h))
    return grid

all_images = [Image1, Image2, Image3, Image4]
grid = image_grid(all_images, rows=1, cols=4)
grid


In [ ]:
!cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install ./diffusers
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt

In [ ]:
!nvidia-smi

In [ ]:
!accelerate config default --mixed_precision fp16

In [ ]:
import os

os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-4'
os.environ['DATASET_NAME'] = f'Ali-fb/martin_valen_dataset'
os.environ['OUTPUT_DIR'] = f'Ed_martin_valen-model-v1-2_400_test'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --use_ema \
  --resolution=256 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision='fp16' \
  --max_train_steps=400 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --checkpointing_steps=100000 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR